## Generate Training File

In [1]:
import json, os, random
RAW_JSON_PATH = os.path.expanduser("LEVIR-MCI-dataset/LevirCCcaptions-v2.json")

with open(RAW_JSON_PATH, "r") as f: 
    raw_captions = json.load(f)

raw_captions = [row for row in raw_captions if row['filepath'] == "train"]

In [2]:
def read_file(file_path):
    with open(file_path, 'r') as file:
        content = file.read()
    return content

In [3]:
collected_Q_for_count = []
for img in raw_captions:
    if img['changeflag'] == 0:
        continue
    file_path = f"count/results/{img['filename'].replace('.png', '.txt')}"
    QA = read_file(file_path).replace('\n\n','\n').split("\n")
    assert(len(QA)==2), file_path
    Q = QA[0].replace("Question: ", "")
    collected_Q_for_count.append(Q)

In [4]:
collected_Q_for_count = list(set(collected_Q_for_count))
collected_Q_for_count = [x for x in collected_Q_for_count if 'road' in x and 'building' in x]

In [16]:
instructs = []
lenList = []
global_id = 0
for img in raw_captions:
    # 1）简单描述
    for cap in img['sentences']:
        rslt = {
            'id': global_id,
            'image': [f"{img['filepath']}/A/{img['filename']}", f"{img['filepath']}/B/{img['filename']}"],
            'changeflag': img['changeflag'],
            'conversations': [],
            'type': 1
            }
        rslt['conversations'].extend([
            {
                "from": "human",
                "value": "<image> <image> Please briefly describe the changes in these two images."
            },
            {
                "from": "gpt",
                "value": cap.strip().replace(" .", ".").capitalize()
            }
        ]
        )
        instructs.append(rslt)
        global_id += 1
    

    # 2）判断是否有变化
    rslt = {
        'id': global_id,
        'image': [f"{img['filepath']}/A/{img['filename']}", f"{img['filepath']}/B/{img['filename']}"],
        'changeflag': img['changeflag'],
        'conversations': [],
        'type': 2
        }
    rslt['conversations'].extend([
        {
            "from": "human",
            "value": "<image> <image> Please judge whether these two images have changed. Please answer yes or no."
        },
        {
            "from": "gpt",
            "value": "Yes" if img['changeflag'] == 1 else "No"
        }
    ]
    )

    instructs.append(rslt)
    global_id += 1

    # 3）判断变化的类别和数量
    rslt = {
        'id': global_id,
        'image': [f"{img['filepath']}/A/{img['filename']}", f"{img['filepath']}/B/{img['filename']}"],
        'changeflag': img['changeflag'],
        'conversations': [],
        'type': 3
        }
    
    def getCountQA(row):
        file_path = f"count/results/{row['filename'].replace('.png', '.txt')}"
        QA = read_file(file_path).replace('\n\n','\n').split("\n")
        assert(len(QA)==2), file_path
        Q = '<image> <image> '+ QA[0].replace("Question: ", "")
        A = QA[1].replace("Answer: ", "")
        if row['changeflag'] == 0:
            Q = '<image> <image> '+ random.choice(collected_Q_for_count)
        return Q, A

    rslt['conversations'].extend([
        {
            "from": "human",
            "value": getCountQA(img)[0]
        },
        {
            "from": "gpt",
            "value": getCountQA(img)[1]
        }
    ]
    )
    instructs.append(rslt)
    global_id += 1

    # 4）GPT提问对话判断变化目标的轮廓
    rslt = {
        'id': global_id,
        'image': [f"{img['filepath']}/A/{img['filename']}", f"{img['filepath']}/B/{img['filename']}"],
        'conversations': [],
        'type': 4
        }
    
    def getCounterAnswer(row):
        ansList = []
        for cat, value in row['contours'].items():
            if row['changeflag'] == 0:
                value = []
            ansList.append(f"The contours of the changed {cat}s are:{json.dumps(value, separators=(',', ':'))}")
        ans = ", ".join(ansList) + "."
        lenList.append(len(ans))
        return ans

    rslt['conversations'].extend([
        {
            "from": "human",
            "value": "<image> <image> Please draw the contours of the changed buildings and roads, approximating them with polygons."
        },
        {
            "from": "gpt",
            "value": getCounterAnswer(img)
        }
    ]
    )
    instructs.append(rslt)
    global_id += 1

    # 5）GPT提问对话
    def getConv(row):
        rslt = []
        file_path = f"conversation/results/{row['filename'].replace('.png', '.txt')}"
        conv = read_file(file_path).replace('\n\n','\n').strip().split("\n")
        assert(len(conv) % 2 == 0), file_path
        for idx in range(len(conv)//2):
            Q = conv[2*idx].replace("Question: ", "")
            A = conv[2*idx + 1].replace("Answer: ", "")
            # if "how many" in Q or "how much" in Q or " Please answer yes or no." in Q:
            #     continue
            rslt.append([
                {
                    "from": "human",
                    "value": '<image> <image> '+ Q
                },
                {
                    "from": "gpt",
                    "value": A
                }
            ])
        return rslt
    convs = getConv(img)
    for c in convs:
        rslt = {
            'id': global_id,
            'image': [f"{img['filepath']}/A/{img['filename']}", f"{img['filepath']}/B/{img['filename']}"],
            'changeflag': img['changeflag'],
            'conversations': [],
            'type': 5
            }
        rslt['conversations'].extend(c)
        instructs.append(rslt)
        global_id += 1
    # 6）多轮对话
    rslt = {
        'id': global_id,
        'image': [f"{img['filepath']}/A/{img['filename']}", f"{img['filepath']}/B/{img['filename']}"],
        'conversations': [],
        'type': 6
        }
    def getNumAnswer(row):
        ans = []
        road_count = row['change_counts']['road']
        building_count = row['change_counts']['building']
        if road_count == 0 or row['changeflag'] == 0 :
            ans = "No roads has changed, "
        elif road_count == 1:
            ans = "1 road has changed, "
        else:
            ans = f"{road_count} roads has changed, "

        if building_count == 0 or row['changeflag'] == 0:
            ans += "and no buildings has changed."
        elif building_count == 1:
            ans += "and 1 building has changed."
        else:
            ans += f"and {building_count} buildings has changed."
        return ans
    
    rslt['conversations'].extend([
        {
            "from": "human",
            "value": "<image> <image> Please judge whether these two images have changed. Please answer yes or no."
        },
        {
            "from": "gpt",
            "value": "Yes" if img['changeflag'] == 1 else "No"
        },
        {
            "from": "human",
            "value": "If changes have occurred, count the number of road and building changes separately."
        },
        {
            "from": "gpt",
            "value": getNumAnswer(img)
        },
        {
            "from": "human",
            "value": "Based on the above analysis, please describe the changes of these two images in detail."
        },
        {
            "from": "gpt",
            "value": random.choice(img['sentences']).strip().replace(" .", ".").capitalize()
        }
    ]
    )
    instructs.append(rslt)
    global_id += 1

In [19]:
print(len([x for x in instructs]))

91678


In [32]:
with open("LEVIR-MCI-dataset/ChangeChat_instruct_gpt_114k.json", "w") as f:
    json.dump(instructs, f)

## Generate Benchmark File

In [1]:
import json, os
RAW_JSON_PATH = os.path.expanduser("~/autodl-tmp/LEVIR-MCI-dataset/LevirCCcaptions-v2.json")
with open(RAW_JSON_PATH, "r") as f: 
    raw_captions = json.load(f)
raw_captions = [row for row in raw_captions if row['filepath'] == "test"]

In [7]:
import jsonlines
instructs = []
global_id = 100001
for img in raw_captions:
    for cap in img['sentences']:
        rslt = {
            'question_id': global_id,
            'image': [f"{img['filepath']}/A/{img['filename']}", f"{img['filepath']}/B/{img['filename']}"],
            'answer': cap.replace(" .", "."),
            'category': 'CC'
            }
        instructs.append(rslt)
        global_id += 1
        break

with jsonlines.open("/root/autodl-tmp/LEVIR-MCI-dataset/Test_CC_gt.jsonl", "w") as wfd:
    for data in instructs:
        wfd.write(data)

In [2]:
import jsonlines
instructs = []
global_id = 100001
for img in raw_captions:
    for cap in img['sentences']:
        rslt = {
            'question_id': global_id,
            'image': [f"{img['filepath']}/A/{img['filename']}", f"{img['filepath']}/B/{img['filename']}"],
            'text': "<image> <image> Please Describe what changes occurred between the two remote sensing images?",
            'changeflag': img['changeflag'],
            'category': 'CC'
            }
        instructs.append(rslt)
        global_id += 1
        break

with jsonlines.open("/root/autodl-tmp/LEVIR-MCI-dataset/Test_CC_v3.jsonl", "w") as wfd:
    for data in instructs:
        wfd.write(data)

In [4]:
import jsonlines
instructs = []
global_id = 100001
for img in raw_captions:
    for cap in img['sentences']:
        rslt = {
            'question_id': global_id,
            'image': [f"{img['filepath']}/A/{img['filename']}", f"{img['filepath']}/B/{img['filename']}"],
            'text': "<image> <image>\n Have the objects in these two remote sensing images changed? Please answer yes or no.",
            'category': 'CC'
            }
        instructs.append(rslt)
        global_id += 1
        break

with jsonlines.open("/root/autodl-tmp/LEVIR-MCI-dataset/Test_CC_v3_conv_r1.jsonl", "w") as wfd:
    for data in instructs:
        wfd.write(data)